In [ ]:
# Standard installs
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets # If loading from Hugging Face Hub

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8nv4vo1_/unsloth_2788e1b8cd1c4b099a70bf9bdf2b0e3d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8nv4vo1_/unsloth_2788e1b8cd1c4b099a70bf9bdf2b0e3d
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 13.5 MB/s eta 0:00:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset

# Load model - Llama 3 8B Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = 4096,
    dtype = torch.float16,
    load_in_4bit = True,
)

# Enable LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

# Load small public corpus
dataset = load_dataset("Abirate/english_quotes", split = "train[:500]")  # Only 500 quotes for fast training

# Preprocessing
def formatting_prompts_func(example):
    prompt = example['quote']  # Just the quote
    return {"text": prompt}

dataset = dataset.map(formatting_prompts_func)

# Tokenization
def tokenize_function(sample):
    model_inputs = tokenizer(
        sample["text"],
        truncation=True,
        padding="max_length",
        max_length=4096,
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(["author", "quote", "text"])

# Ready to Train
FastLanguageModel.for_training(model)

from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model = model,
    train_dataset = dataset,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,  # Slightly more steps to show learning
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs_continued_pretrain",
        optim = "paged_adamw_32bit",
        fp16 = True,
        bf16 = False,
    ),
)

trainer.train()

# Save the model
model.save_pretrained("llama3_continued_pretrain_unsloth")
tokenizer.save_pretrained("llama3_continued_pretrain_unsloth")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jayanth-kalyanam (jayanth-kalyanam-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,15.331700
2,15.561300
3,15.526500
4,15.552900
5,15.148300
6,14.392900
7,13.208400
8,12.000600
9,11.531800
10,10.530000


('llama3_continued_pretrain_unsloth/tokenizer_config.json',
 'llama3_continued_pretrain_unsloth/special_tokens_map.json',
 'llama3_continued_pretrain_unsloth/tokenizer.json')